**Preprocessing**

In [1]:
!pip uninstall -y numpy gensim
!pip install --no-cache-dir numpy==1.23.5 gensim

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 154.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 135.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 168.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 112.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
bigframes 2.3.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.6 requires numpy>=1.24.4, but you have numpy 1.2

In [1]:
import re
import numpy as np
import gensim.downloader as api
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

word2vec_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [6]:
import pandas as pd
df = pd.read_csv('Combined Data.csv')
def preprocess_and_tokenize(text):
    if not isinstance(text, str):
        return []

    text = re.sub(r"[^A-Za-z0-9\s]", "", text.lower())
    tokens = simple_preprocess(text, deacc=True)
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
    return tokens

df['tokens'] = df['statement'].apply(preprocess_and_tokenize)
def get_average_word2vec(tokens, model, vector_size=300):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(vector_size)

df['word2vec_vector'] = df['tokens'].apply(lambda x: get_average_word2vec(x, word2vec_model))
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['status'] = le.fit_transform(df['status'])

label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label Mapping:", label_mapping)

print(df['status'].value_counts())


Label Mapping: {'Anxiety': 0, 'Bipolar': 1, 'Depression': 2, 'Normal': 3, 'Personality disorder': 4, 'Stress': 5, 'Suicidal': 6}
status
3    16351
2    15404
6    10653
0     3888
1     2877
5     2669
4     1201
Name: count, dtype: int64


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare data
X = np.stack(df['word2vec_vector'].values)
y = df['status'].values

X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [8]:
device

device(type='cuda')

LSTM + DT

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LSTMFeatureExtractor(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.5):
        super(LSTMFeatureExtractor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

class SoftDecisionTree(nn.Module):
    def __init__(self, input_dim, num_classes, num_nodes=10):
        super(SoftDecisionTree, self).__init__()
        self.inner_nodes = nn.Linear(input_dim, num_nodes)
        self.leaf_nodes = nn.Linear(num_nodes, num_classes)

    def forward(self, x):
        decision_weights = torch.sigmoid(self.inner_nodes(x))
        leaf_outputs = self.leaf_nodes(decision_weights)
        return leaf_outputs

class LSTMDecisionTree(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_nodes=10):
        super(LSTMDecisionTree, self).__init__()
        self.feature_extractor = LSTMFeatureExtractor(input_dim, hidden_size=hidden_dim)
        self.decision_tree = SoftDecisionTree(hidden_dim, num_classes, num_nodes)

    def forward(self, x):
        features = self.feature_extractor(x)
        output = self.decision_tree(features)
        return output

class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

batch_size = 64
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = X_train.shape[1]
num_classes = len(torch.unique(torch.tensor(y_train)))
model = LSTMDecisionTree(input_dim=input_dim, hidden_dim=128, num_classes=num_classes, num_nodes=10)

# Train the model
train_model(model, train_loader, test_loader, epochs=10, lr=0.001)


<ipython-input-15-072f0c1e1c9b>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)
<ipython-input-15-072f0c1e1c9b>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype=torch.long)
<ipython-input-15-072f0c1e1c9b>:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  num_classes = len(torch.unique(torch.tensor(y_train)))


Epoch 1, Loss: 1.4243, Train Accuracy: 46.86%
Epoch 2, Loss: 1.1035, Train Accuracy: 56.52%
Epoch 3, Loss: 1.0242, Train Accuracy: 60.57%
Epoch 4, Loss: 0.9571, Train Accuracy: 64.51%
Epoch 5, Loss: 0.9026, Train Accuracy: 66.98%
Epoch 6, Loss: 0.8691, Train Accuracy: 68.06%
Epoch 7, Loss: 0.8325, Train Accuracy: 69.30%
Epoch 8, Loss: 0.8089, Train Accuracy: 70.07%
Epoch 9, Loss: 0.7894, Train Accuracy: 70.71%
Epoch 10, Loss: 0.7757, Train Accuracy: 71.09%
Test Accuracy: 71.74%


LSTM + SVM

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LSTMFeatureExtractor(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.5):
        super(LSTMFeatureExtractor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

class SoftSVM(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SoftSVM, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

class LSTMSVM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(LSTMSVM, self).__init__()
        self.feature_extractor = LSTMFeatureExtractor(input_dim, hidden_dim)
        self.svm = SoftSVM(hidden_dim, num_classes)

    def forward(self, x):
        features = self.feature_extractor(x)
        output = self.svm(features)
        return output

class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

batch_size = 64
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = X_train.shape[1]
num_classes = len(torch.unique(y_train))
model = LSTMSVM(input_dim=input_dim, hidden_dim=128, num_classes=num_classes)

train_model(model, train_loader, test_loader, epochs=10, lr=0.001)

<ipython-input-16-f0821f85474b>:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)
<ipython-input-16-f0821f85474b>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype=torch.long)


Epoch 1, Loss: 1.2052, Train Accuracy: 53.39%
Epoch 2, Loss: 0.9841, Train Accuracy: 63.14%
Epoch 3, Loss: 0.9004, Train Accuracy: 66.25%
Epoch 4, Loss: 0.8494, Train Accuracy: 68.16%
Epoch 5, Loss: 0.8134, Train Accuracy: 69.19%
Epoch 6, Loss: 0.7872, Train Accuracy: 69.98%
Epoch 7, Loss: 0.7722, Train Accuracy: 70.57%
Epoch 8, Loss: 0.7586, Train Accuracy: 71.33%
Epoch 9, Loss: 0.7458, Train Accuracy: 71.59%
Epoch 10, Loss: 0.7361, Train Accuracy: 71.71%
Test Accuracy: 72.50%


LSTM+RF

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
from sklearn.ensemble import RandomForestClassifier
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LSTMFeatureExtractor(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.5):
        super(LSTMFeatureExtractor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = X.clone().detach().float() if isinstance(X, torch.Tensor) else torch.tensor(X, dtype=torch.float32)
        self.y = y.clone().detach().long() if isinstance(y, torch.Tensor) else torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def extract_features(model, data_loader):
    model.to(device)
    model.eval()
    features, labels = [], []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            batch_features = model(X_batch)
            features.append(batch_features.cpu())
            labels.append(y_batch.cpu())

    return torch.cat(features).numpy(), torch.cat(labels).numpy()

batch_size = 64
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = X_train.shape[1]
lstm_model = LSTMFeatureExtractor(input_size=input_dim, hidden_size=128)

X_train_feats, y_train_feats = extract_features(lstm_model, train_loader)
X_test_feats, y_test_feats = extract_features(lstm_model, test_loader)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_feats, y_train_feats)
y_pred = rf_model.predict(X_test_feats)

accuracy = np.mean(y_pred == y_test_feats) * 100
print(f"Random Forest Test Accuracy: {accuracy:.2f}%")


Random Forest Test Accuracy: 65.87%
